In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from os import mkdir
from os.path import join, exists

from scipy.stats import ttest_ind

from constants import N_TARGETS, N_SHADOWS, DEEPSETS_HYPERPARAMS, MLP_HYPERPARAMS

In [2]:
import requests

class Barmecide:
    def __init__(self, token, chat=297134477):
        self.token = token
        self.chat = chat

    def send_message(self, text):
        requests.post(
            'https://api.telegram.org/' + 'bot{}/sendMessage'.format(self.token),
            params=dict(chat_id=self.chat, text=text)
        )
        
barmecide = Barmecide('2136899889:AAGglINBJ-4ZF5AY6npPDzVt9pqfdoqn_1A')

In [3]:
RESULTS_FOLDER = os.path.join(os.getcwd(), 'results')
FIGURES_FOLDER = os.path.join(os.getcwd(), 'figures')
DATA_FOLDER = os.path.join(os.getcwd(), 'data')

In [4]:
if not exists(RESULTS_FOLDER):
    mkdir(RESULTS_FOLDER)
    
if not exists(FIGURES_FOLDER):
    mkdir(FIGURES_FOLDER)

In [5]:
sns.set_theme(palette=sns.color_palette('YlOrRd_r', 3))
sns.set_context('notebook', font_scale=1.5)

In [6]:
asia = pd.read_csv(join(DATA_FOLDER, 'asia.csv.gz'), index_col=0)
print(asia.shape)
asia.head()

(16384, 8)


,asia,tub,smoke,lung,bronc,either,xray,dysp
0,1,1,0,1,0,1,1,0
1,1,1,1,0,0,0,0,0
2,1,1,0,1,0,1,1,0
3,1,1,1,1,0,1,1,0
4,1,1,0,1,0,1,0,0


In [7]:
asia.dtypes

asia      int64
tub       int64
smoke     int64
lung      int64
bronc     int64
either    int64
xray      int64
dysp      int64
dtype: object

In [8]:
asia.xray.value_counts()

1    14541
0     1843
Name: xray, dtype: int64

In [9]:
asia.dysp.value_counts()

1    9237
0    7147
Name: dysp, dtype: int64

In [10]:
asia.smoke.value_counts()

0    8290
1    8094
Name: smoke, dtype: int64

In [ ]:
%%time

from propinfer.generator import SubsamplingGenerator
from propinfer.experiment import Experiment
from propinfer.model import MLP

from models import IRM

from copy import copy

MLP_HYPERPARAMS['input_size'] = asia.shape[1]
MLP_HYPERPARAMS['n_classes'] = 2

N_RUNS = 10

res = pd.DataFrame(columns=['Dataset', 'Model', 'Proportion', 'Accuracy'])

for prop in np.arange(0.1, 1., 0.1):
    print(f'Running ERM experiment for prop={prop:.1f}')
    gen = SubsamplingGenerator(data=asia, label_col='xray', sensitive_attribute='dysp', proportion=prop, split=True)
    exp = Experiment(generator=gen, label_col='xray', model=MLP, n_targets=N_TARGETS, n_shadows=N_SHADOWS, hyperparams=MLP_HYPERPARAMS, n_queries=2048)
    
    exp.run_targets()
    exp.run_shadows()
    
    acc = exp.run_whitebox_deepsets(DEEPSETS_HYPERPARAMS, n_outputs=N_RUNS)
    
    res.loc[len(res)] = ('Asia', 'ERM', prop, acc)
    
    barmecide.send_message(f'Experiment done: ERM prop={prop} (res={np.mean(acc)})')
    
IRM_HYPERPARAMS = copy(MLP_HYPERPARAMS)    

IRM_HYPERPARAMS['input_size'] = asia.shape[1] - 2
IRM_HYPERPARAMS['epochs'] = 2**11
IRM_HYPERPARAMS['reg'] = 0.1
IRM_HYPERPARAMS['env_label'] = 'smoke'
    
for prop in np.arange(0.1, 1., 0.1):
    print(f'Running IRM experiment for prop={prop:.1f}')
    gen = SubsamplingGenerator(data=asia, label_col='xray', sensitive_attribute='dysp', proportion=prop, split=True)
    exp = Experiment(generator=gen, label_col='xray', model=IRM, n_targets=N_TARGETS, n_shadows=N_SHADOWS, hyperparams=IRM_HYPERPARAMS, n_queries=2048)
    
    exp.run_targets()
    exp.run_shadows()
    
    acc = exp.run_whitebox_deepsets(DEEPSETS_HYPERPARAMS, n_outputs=N_RUNS)
    
    res.loc[len(res)] = ('Asia', 'IRM', prop, acc)
    
    barmecide.send_message(f'Experiment done: IRM prop={prop} (res={np.mean(acc)})')

Running ERM experiment for prop=0.1


In [ ]:
res.to_csv('res.csv')
res

In [ ]:
pd.read_csv('res.csv')